# Install Packages

In [1]:
# Install WhisperX
! pip install git+https://github.com/m-bain/whisperx.git


  Cloning https://github.com/m-bain/whisperx.git to /tmp/pip-req-build-9g1lcqxe
  Running command git clone --filter=blob:none --quiet https://github.com/m-bain/whisperx.git /tmp/pip-req-build-9g1lcqxe
  Resolved https://github.com/m-bain/whisperx.git to commit f2da2f858e99e4211fe4f64b5f2938b007827e17
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 930.3 kB/s eta 0:00:00 0:00:01
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 2.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 7.7 MB/s eta 0:00:00
     ━

In [2]:
!pip install dlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 10.1 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for dlib: filename=dlib-19.24.4-cp310-cp310-linux_x86_64.whl size=3327190 sha256=51c867480855ddd404b887ee0762183fd1cca189e2b77d616502bb2d26d27f8a
  Stored in directory: /root/.cache/pip/wheels/08/5e/55/a7db8d57920eb2c2aa41c36dc4c6d3cd12323865de191a7211
Successfully built dlib


In [3]:
!pip install moviepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 2.3 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 51.6 MB/s eta 0:00:00:00:0100:01
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110720 sha256=c769b08598da2908c9f233e17ff3793a8c8916688c376b22465a2aebe8e5774f
  Stored in directory: /root/.cache/pip/wheels/96/32/2d/e10123bd88fbfc02fed53cc18c80a171d3c87479ed845fa7c1
Successfully built moviepy
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
    Uninstalling decorator-5.1.1:
      Successfully uninstalled decorator-5.1.1


# Checking

In [4]:
# import cv2
# import matplotlib.pyplot as plt
# video_path = "/kaggle/input/fakeavceleb/FakeAVCeleb_v1.2/FakeAVCeleb_v1.2/FakeVideo-FakeAudio/Caucasian (European)/men/id03205/00150_id00519_wavtolip.mp4"

# # if not cv2.VideoCapture(video_path).isOpened():
# #     print("Error: Could not open video.")

# # Open the video file
# cap = cv2.VideoCapture(video_path)
# fps = cap.get(cv2.CAP_PROP_FPS)
# print("FPS: ", fps)
# # Check if the video was opened successfully
# if not cap.isOpened():
#     print("Error: Could not open video.")
#     exit()


# # Function to display frame using Matplotlib
# def display_frame(frame):
#     # Convert the frame from BGR to RGB
#     frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     plt.imshow(frame_rgb)
#     plt.axis('off')  # Hide axes
#     plt.show()

# # Loop to read and display frames
# while True:
#     ret, frame = cap.read()

#     # Break the loop if there are no more frames to read
#     if not ret:
#         break

#     # Display the frame using Matplotlib
#     display_frame(frame)

#     # Add a delay to control frame display rate (e.g., 25 milliseconds)
#     plt.pause(0.025)

# # Release the video capture object
# cap.release()

# Preprocessing


## Import Packages

In [5]:
import librosa
import moviepy.editor as mp
import tempfile
import numpy as np

import cv2
import dlib

import whisperx
import gc

2024-06-26 11:37:37.864165: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-26 11:37:37.864304: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-26 11:37:38.142476: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Word Alignment Processing


In [6]:
# CPU
# device = "cpu" # cuda
# batch_size = 16
# compute_type = "float32" #16
import whisperx

# GPU
device = "cuda" 
batch_size = 16
compute_type = "float16"

# Speech to text transcription model
model = whisperx.load_model("large-v2", device, compute_type=compute_type)

def getWordAlignment(audio_file):
    audio = whisperx.load_audio(audio_file)
    result = model.transcribe(audio, batch_size=batch_size)
    
    if (result["language"] != "en"):
        return None
    
    # Word alignment model
    word_model, metadata = whisperx.load_align_model(language_code = result["language"], device = device)
    
    result = whisperx.align(result["segments"],
                        word_model,
                        metadata,
                        audio,
                        device,
                        return_char_alignments = False)

    print(result["segments"])
    return result

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.80k [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

No language specified, language will be first be detected for each audio file (increases inference time).


100%|█████████████████████████████████████| 16.9M/16.9M [00:01<00:00, 15.2MiB/s]


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.1.2. Bad things might happen unless you revert torch to 1.x.


In [7]:
# Get start and end timestamp for each word in a MP4 file
def getTimestamp(result):
    word_timestamp = []

    for item in result["segments"]:
        for word in item.get("words"):
            start_end_timestamp = [word.get("start"),word.get("end")]
            word_timestamp.append(start_end_timestamp)

    return word_timestamp

## Video Processing


In [8]:
face_detector = dlib.get_frontal_face_detector()

dlib_facelandmark = dlib.shape_predictor("/kaggle/input/dlib-face-landmarks/shape_predictor_68_face_landmarks.dat")

# Define fixed size for the mouth ROI
fixed_width = 100
fixed_height = 50

def mouth_extractor(cap, start_timestamp, end_timestamp, fps):
    # Get frame rate
    start_frame = int(start_timestamp * fps)
    end_frame = int(end_timestamp * fps)

    # Set the starting frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    processed_frame = [] # final processed frame

    missing_frame_thres = (end_frame - start_frame) / 2 # threshold for termination based on missing frame

    missing_frame = 0 # number of missing frame

    first_frame = True
    num_face_registered = 0 # number of face detected across frame

    all_face = {} # Save all extracted mouth region for each person in the video
    mouth_open_close_series = {} # Save the difference between mouth top (y) and mouth bottom (y) to determine which person is speaking

    for frame_number in range(start_frame, end_frame+1):
        ret, frame = cap.read()

        if not ret:
            print("Failed to read frame from video.")
            return []

        if (missing_frame >= missing_frame_thres):
            print("Too many missing frame at this section")
            return []

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = face_detector(gray)

        num_face_detected = len(faces)

        # Check if no faces were detected
        if num_face_detected == 0:
            missing_frame = missing_frame + 1
            continue

        # detected new face in the following frame (after first frame), register new mouth x coordinate
        if (num_face_detected > num_face_registered) and not first_frame:

            # repeat the process by the number of new face detected
            for i in range(num_face_detected - num_face_registered):
                mouth_distance = {}
                
                for face in faces:
                    # get face landmarks
                    landmarks = dlib_facelandmark(gray, face)

                    # extract mouth landmark at 48 as ID
                    mouth_x = landmarks.part(48).x - 15
                    
                    # Find the nearest mouth in the registered face
                    nearest_distance = [0,100000]
                    for registered_mouth in all_face:
                        # Find the distance between two mouth in x coordinate
                        difference = abs(registered_mouth - mouth_x)
                        
                        # Identify the nearest mouth
                        if (difference < nearest_distance[1]):
                            nearest_distance = [mouth_x, difference]
                    
                    mouth_distance[nearest_distance[0]] = nearest_distance[1]
                
                farthest = [0,0]
                # Find the mouth with the largest difference in distance
                for key, distance in mouth_distance.items():
                    if distance > farthest[1]:
                        farthest[0] = key
                        farthest[1] = distance
                
                # register new mouth with empty list
                all_face[farthest[0]] = [] # mouth region
                mouth_open_close_series[farthest[0]] = 0 # difference between top lip (y) and bottom lip (y)

            # update number of face registered
            num_face_registered = num_face_detected


        for face in faces:
            # get face landmarks
            landmarks = dlib_facelandmark(gray, face)

            # extract mouth region
            mouth_x = landmarks.part(48).x - 15
            mouth_y = landmarks.part(51).y - 15
            mouth_w = landmarks.part(54).x - mouth_x + 15
            mouth_h = landmarks.part(57).y - mouth_y + 15


            # When first frame, register the mouth x coordinate to the dictionary
            if first_frame:
                num_face_registered = num_face_detected
                all_face[mouth_x] = [gray[mouth_y:mouth_y + mouth_h, mouth_x:mouth_x + mouth_w]] # mouth region
                mouth_open_close_series[mouth_x] = abs(landmarks.part(63).y - landmarks.part(67).y) # difference between top lip (y) and bottom lip (y)


            else:
                # Append to the nearest mouth_x list
                nearest_x = [99999,0]

                for mouth in all_face:
                    difference = abs(mouth - mouth_x)
                    if (difference < nearest_x[0]):
                        nearest_x[0] = difference # difference between two mouth x coordinate
                        nearest_x[1] = mouth # key

                # Append the frame to the nearest mouth (key)
                all_face[nearest_x[1]].append(gray[mouth_y:mouth_y + mouth_h, mouth_x:mouth_x + mouth_w]) # mouth region
                mouth_open_close_series[nearest_x[1]] = mouth_open_close_series[nearest_x[1]] + abs(landmarks.part(63).y - landmarks.part(67).y) # add the difference between top lip (y) and bottom lip (y)


                # update the mouth x coordinate (key) with the latest position
                all_face[mouth_x] = all_face.pop(nearest_x[1])
                mouth_open_close_series[mouth_x] = mouth_open_close_series.pop(nearest_x[1])

        first_frame = False
    
    # Determine the current speaker by the difference between mouth top and bottom position across frame
    # The speaker will have the largest the difference across frame because they will open and close their mouth
    speaker_key = max(mouth_open_close_series, key=mouth_open_close_series.get)

    # check number of frame. Return empty list when too many missing frame
    if len(all_face[speaker_key]) <= missing_frame_thres:
        return []

    # Resize frame
    for mouth_roi in all_face[speaker_key]:
        if mouth_roi.shape[1] < fixed_width or mouth_roi.shape[0] < fixed_height:
            interpolation_set = cv2.INTER_CUBIC # For upsampling
        else:
            interpolation_set = cv2.INTER_AREA # For downsampling

        resized_mouth_roi = cv2.resize(mouth_roi, (fixed_width, fixed_height), interpolation=interpolation_set)

        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
        normalized_frame = resized_mouth_roi / 255

        processed_frame.append(normalized_frame)

    return processed_frame



## Audio Processing


In [9]:
def generate_spectrogram(y, sr, start_time, end_time):
    
    # Extract the desired clip
    clip = y[int(start_time*sr):int(end_time*sr)]
    
    # Compute the spectrogram
    Mel_spectrogram = librosa.feature.melspectrogram(y=clip)
    
    log_spectrogram = librosa.power_to_db(Mel_spectrogram, ref=np.max)

        
    return log_spectrogram


## Start preprocessing

In [10]:
# Create folder to output processed data
import os

folder_path = '/kaggle/working/'

for item in ["FakeVideo-FakeAudio","FakeVideo-RealAudio","RealVideo-FakeAudio","RealVideo-RealAudio"]:
    temp_path = folder_path+item
    
    if not os.path.exists(temp_path):
        os.mkdir(temp_path)


In [12]:
import os
import random
path = "/kaggle/input/fakeavceleb/FakeAVCeleb_v1.2/FakeAVCeleb_v1.2/"
dataset = os.listdir(path)

final_dataset = []

# access 4 main folder (FakeVideo-FakeAudio, FakeVideo-RealAudio, RealVideo-RealAudio, RealVideo-FakeAudio)
for folder in dataset:
    if os.path.isdir(path+folder):

        # access 5 main area - African, Asian (East),Asian (South), Caucasian(US),Caucasian (EU) 
        sub_folder = os.listdir(path+folder)
        for area in sub_folder:

            # access gender folder - male, female
            area_folder = os.listdir(path+folder+"/"+area)
            for gender in area_folder:

                video_folder = os.listdir(path+folder+"/"+area+"/"+gender)

                # Randomly sample 5 video folder from the folder                
                selected_video_folder = random.sample(video_folder, 5)

                # pick one video from each video folder, ignore non-mp4 file
                for selected_folder in selected_video_folder:


                    videos = os.listdir(path+folder+"/"+area+"/"+gender+"/"+selected_folder)

                    for file in videos:

                        video_path = path+folder+"/"+area+"/"+gender+"/"+selected_folder+"/"+file

                        #ignore non-mp4 file and video that cannot be opened 
                        if not file.endswith(".mp4") or not cv2.VideoCapture(video_path).isOpened():
                            continue

                        final_dataset.append(video_path)

                        #Only load one video inside the folder
                        break;




In [ ]:
print(len(final_dataset))


In [ ]:
final_dataset[0].split("/")

## Zip folder to download

In [ ]:
counter = 0
total_video = len(final_dataset)

for video_path in final_dataset:

    counter = counter +1
    
    print("Processing File: ", video_path, f"| Progress---> {counter}/{total_video}")

    # load the video file
    video = mp.VideoFileClip(video_path)

    audio = video.audio

    # Save the audio as a temporary WAV file
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=True) as tmp_audio_file:
        audio.write_audiofile(tmp_audio_file.name)
        tmp_audio = tmp_audio_file.name

        # get word alignment result
        alignment_result = getWordAlignment(tmp_audio)

        # Load the audio using librosa
        y, sr = librosa.load(tmp_audio)

    if not alignment_result:
        counter = counter - 1
        continue 

    # Get start and end timestamp for each word in the audio file
    word_timestamp = getTimestamp(alignment_result)


    for timestamp in word_timestamp:
        # ensure the timestamp is not empty
        if (timestamp[0] == None ):
            continue

        print(f"Timestamp -> {timestamp[0]} - {timestamp[1]}")
        # Read video MP4 File
        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        print("FPS: ", fps)

        # timestamp[0] -> start time | timestamp[1] -> end time
        processed_video = mouth_extractor(cap, timestamp[0], timestamp[1], fps)

        # Process and convert audio data to log mel spectrogram
        processed_audio = generate_spectrogram(y, sr, timestamp[0], timestamp[1])
        print('Shape: ', processed_audio.shape)

        if processed_audio.shape[1] > 25:
            print("outlier: Audio frame > 25")
            continue 

        if not processed_video:
            print("empty frame")
            continue

        # obtain the filename
        temp = video_path.split("/")
        folder = temp[6]
        filename = os.path.splitext(temp[-1])[0]

        # Create new folder for the data
        folder_path = f"/kaggle/working/{folder}/{filename}_{timestamp[0]}_{timestamp[1]}/"
        os.mkdir(folder_path)

        np.save(folder_path+"video", processed_video)
        np.save(folder_path+"audio", processed_audio)

        print("="*100)
        print(f"Saved Data")
        print("="*100)
    
    cap.release()

Processing File:  /kaggle/input/fakeavceleb/FakeAVCeleb_v1.2/FakeAVCeleb_v1.2/FakeVideo-FakeAudio/African/men/id00987/00160_id01207_wavtolip.mp4 | Progress---> 1/200
MoviePy - Writing audio in /tmp/tmpb3y48mgg.wav


MoviePy - Done.


Detected language: en (0.99) in first 30s of audio...


Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_fairseq_base_ls960_asr_ls960.pth" to /root/.cache/torch/hub/checkpoints/wav2vec2_fairseq_base_ls960_asr_ls960.pth
100%|██████████| 360M/360M [00:01<00:00, 243MB/s]  


[{'start': 0.129, 'end': 4.075, 'text': " Awesome, then there's one in fire all these great records when you start seeing the same name.", 'words': [{'word': 'Awesome,', 'start': 0.129, 'end': 0.39, 'score': 0.624}, {'word': 'then', 'start': 0.43, 'end': 0.65, 'score': 0.768}, {'word': "there's", 'start': 0.69, 'end': 0.91, 'score': 0.853}, {'word': 'one', 'start': 1.01, 'end': 1.111, 'score': 0.836}, {'word': 'in', 'start': 1.151, 'end': 1.231, 'score': 0.362}, {'word': 'fire', 'start': 1.271, 'end': 1.571, 'score': 0.82}, {'word': 'all', 'start': 1.651, 'end': 1.771, 'score': 0.736}, {'word': 'these', 'start': 1.812, 'end': 1.952, 'score': 0.888}, {'word': 'great', 'start': 2.012, 'end': 2.212, 'score': 0.923}, {'word': 'records', 'start': 2.272, 'end': 2.593, 'score': 0.879}, {'word': 'when', 'start': 2.653, 'end': 2.773, 'score': 0.996}, {'word': 'you', 'start': 2.813, 'end': 2.953, 'score': 0.964}, {'word': 'start', 'start': 2.993, 'end': 3.254, 'score': 0.943}, {'word': 'seeing',

...

Detected language: en (0.98) in first 30s of audio...
[{'start': 0.069, 'end': 3.816, 'text': " Comment and let me see how easy it is to get something that's going to Cold Stone.", 'words': [{'word': 'Comment', 'start': 0.069, 'end': 0.47, 'score': 0.626}, {'word': 'and', 'start': 0.51, 'end': 0.61, 'score': 0.325}, {'word': 'let', 'start': 0.65, 'end': 0.81, 'score': 0.92}, {'word': 'me', 'start': 0.851, 'end': 0.951, 'score': 0.831}, {'word': 'see', 'start': 0.991, 'end': 1.171, 'score': 0.806}, {'word': 'how', 'start': 1.211, 'end': 1.391, 'score': 0.964}, {'word': 'easy', 'start': 1.452, 'end': 1.652, 'score': 0.276}, {'word': 'it', 'start': 1.672, 'end': 1.712, 'score': 0.169}, {'word': 'is', 'start': 1.772, 'end': 1.872, 'score': 0.661}, {'word': 'to', 'start': 1.892, 'end': 2.013, 'score': 0.514}, {'word': 'get', 'start': 2.033, 'end': 2.193, 'score': 0.81}, {'word': 'something', 'start': 2.233, 'end': 2.574, 'score': 0.916}, {'word': "that's", 'start': 2.614, 'end': 2.854, 'sco

MoviePy - Done.


Detected language: en (0.99) in first 30s of audio...
[{'start': 0.049, 'end': 1.03, 'text': ' Yeah, I do.', 'words': [{'word': 'Yeah,', 'start': 0.049, 'end': 0.51, 'score': 0.698}, {'word': 'I', 'start': 0.77, 'end': 0.85, 'score': 0.865}, {'word': 'do.', 'start': 0.87, 'end': 1.03, 'score': 0.826}]}, {'start': 1.071, 'end': 7.139, 'text': 'I mean, people sort of industry, the Clinton, you know, they just when we pray out with time, they just get, you know, sometimes I just talk to me like.', 'words': [{'word': 'I', 'start': 1.071, 'end': 1.131, 'score': 0.94}, {'word': 'mean,', 'start': 1.171, 'end': 1.331, 'score': 0.925}, {'word': 'people', 'start': 1.371, 'end': 1.631, 'score': 0.889}, {'word': 'sort', 'start': 1.651, 'end': 1.852, 'score': 0.847}, {'word': 'of', 'start': 1.872, 'end': 1.932, 'score': 0.727}, {'word': 'industry,', 'start': 1.992, 'end': 2.393, 'score': 0.837}, {'word': 'the', 'start': 2.473, 'end': 2.573, 'score': 0.35}, {'word': 'Clinton,', 'start': 2.593, 'end'

MoviePy - Done.


Detected language: en (1.00) in first 30s of audio...


In [ ]:
!zip -r FakeAVCeleb_GS_Validation.zip /kaggle/working

In [ ]:
# import shutil

# def remove_directory_and_contents(directory_path):
#     try:
#         shutil.rmtree(directory_path)
#         print(f"Successfully removed the directory and all its contents: {directory_path}")
#     except Exception as e:
#         print(f"Failed to remove the directory {directory_path}. Reason: {e}")

# # Example usage
# directory_path = "/kaggle/working/"
# remove_directory_and_contents(directory_path)

## Read processed FakeAVCeleb

In [ ]:
# Read Dataset
base_path = '/kaggle/input/fakeavceleb-validation'

video_data = []
audio_data = []
labels = []

for main_folder in os.listdir(base_path):
    
    temp_path = os.path.join(base_path, main_folder,'content')
    
    for sub_folder in os.listdir(temp_path):
        
        temp_path_2 = os.path.join(temp_path, sub_folder)
        print(sub_folder)
        for video_audio_folder in os.listdir(temp_path_2):
            video_data.append(np.load(os.path.join(temp_path_2, video_audio_folder, "video.npy")))
            audio_data.append(np.load(os.path.join(temp_path_2, video_audio_folder, "audio.npy")))
            
            if 'Fake' in sub_folder:
                labels.append('Fake')
            else:
                labels.append('Real')





In [ ]:
# Initialize padded video data
video_data = pad_sequences(video_data, maxlen = 19,dtype='float32', padding='post', truncating='post', value=0.0 )

# Add a channel dimension
video_data = np.expand_dims(video_data, axis=-1)

In [ ]:
# Find global min and max of log mel spectrogram for normalziation purpose
global_min = min(np.min(spectrogram) for spectrogram in audio_data)
global_max = max(np.max(spectrogram) for spectrogram in audio_data)

for i, audio in enumerate(audio_data):
    # transpose audio data
    transpose_audio = audio.transpose()
    
    # Normalize the log-mel spectrogram
    normalized_audio = (transpose_audio - global_min) / (global_max - global_min)
    
    audio_data[i] = normalized_audio
    

# add paddding to audio data
audio_data = pad_sequences(audio_data, maxlen = 25, dtype='float32', padding='post', truncating='post', value=0.0 )

In [ ]:
video_data = np.array(video_data)
audio_data = np.array(audio_data)
encoded_labels = LabelEncoder().fit_transform(labels)


In [ ]:
# Save unsplited data for cross_validation
np.savez_compressed('Processed_FakeAVCeleb.npz', 
                   video_data=video_data, 
                   audio_data=audio_data, 
                   encoded_labels=encoded_labels)